In [1]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [1,257 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubun

'/content/spark-3.3.1-bin-hadoop3'

In [2]:
from google.colab import drive

drive.mount("/content/gdrive") 
!pwd

Mounted at /content/gdrive
/content


In [3]:
#put your own path in google drive
%cd "/content/gdrive/MyDrive/IA/Modulo1_BigData"
!ls

/content/gdrive/MyDrive/IA/Modulo1_BigData
Entregable.ipynb	 spark-3.3.1-bin-hadoop3.tgz
spark-3.3.1-bin-hadoop3  WIKI_PRICES.csv


## Crear la sesión de trabajo

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import last

In [5]:
#Verificar la funcionalidad de Pyspark 

spark_session = SparkSession.builder.appName('Entregable').getOrCreate()
spark_session

## Cargar los datos

In [6]:
df = spark_session.read.csv('WIKI_PRICES.csv', header = True, inferSchema = True)

In [7]:
df.show()

+------+-------------------+-----+-----+-----+-----+---------+-----------+-----------+---------------+---------------+---------------+---------------+----------+
|ticker|               date| open| high|  low|close|   volume|ex-dividend|split_ratio|       adj_open|       adj_high|        adj_low|      adj_close|adj_volume|
+------+-------------------+-----+-----+-----+-----+---------+-----------+-----------+---------------+---------------+---------------+---------------+----------+
|     A|1999-11-18 00:00:00| 45.5| 50.0| 40.0| 44.0|4.47399E7|        0.0|        1.0|31.041951216877|34.112034304261|27.289627443409|30.018590187749| 4.47399E7|
|     A|1999-11-19 00:00:00|42.94| 43.0|39.81|40.38|1.08971E7|        0.0|        1.0|29.295415060499|29.336349501664|27.160001713052|27.548878904121| 1.08971E7|
|     A|1999-11-22 00:00:00|41.31| 44.0|40.06| 44.0|4705200.0|        0.0|        1.0| 28.18336274218|30.018590187749|27.330561884574|30.018590187749| 4705200.0|
|     A|1999-11-23 00:00:00|

In [8]:
df.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- ex-dividend: double (nullable = true)
 |-- split_ratio: double (nullable = true)
 |-- adj_open: double (nullable = true)
 |-- adj_high: double (nullable = true)
 |-- adj_low: double (nullable = true)
 |-- adj_close: double (nullable = true)
 |-- adj_volume: double (nullable = true)



In [9]:
df = df.groupBy("Ticker").avg("open","close","high","low")

In [10]:
df.show()

+------+------------------+------------------+------------------+------------------+
|Ticker|         avg(open)|        avg(close)|         avg(high)|          avg(low)|
+------+------------------+------------------+------------------+------------------+
|  ACFN| 5.159595073641425| 5.167712239715598|  5.31348743863215|   5.0101387675639|
|  ALXN|61.846555155659544| 61.80857145942056| 62.94784869533896| 60.62664041749151|
|   AAT| 33.56090773053562| 33.56303699613469|33.872553837658714| 33.23921960242961|
|  ABMD|24.849846952245322|24.871593335058844|25.358382593503315|24.334952478322702|
|   AIV| 33.79793227104989| 33.80576316672244| 34.12653463602813|33.449152063066016|
|  ARAY| 7.613426659528919|7.6105299785867215|7.7798464668094205| 7.438157316202708|
|  ACCO|11.012743428931586|11.017502363693659| 11.20868480932873|10.824503907973531|
|   ALE|31.721052625816604|31.733457472187798|31.977249296309214|31.460719789863937|
|  ALSN|28.975527290705397|28.974222148978274| 29.27270527356622|

## Modelo de regresión
MLlib

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [20]:
# Usar la librería de VectorAssembler a las variables indenpendientes
from pyspark.ml.feature import VectorAssembler

featassembler = VectorAssembler(inputCols=['avg(open)','avg(high)','avg(low)'], outputCol = "Independent")
featassembler

VectorAssembler_57f133888452

In [21]:
result = featassembler.transform(df)
result.show()

+------+------------------+------------------+------------------+------------------+--------------------+
|Ticker|         avg(open)|        avg(close)|         avg(high)|          avg(low)|         Independent|
+------+------------------+------------------+------------------+------------------+--------------------+
|  ACFN| 5.159595073641425| 5.167712239715598|  5.31348743863215|   5.0101387675639|[5.15959507364142...|
|  ALXN|61.846555155659544| 61.80857145942056| 62.94784869533896| 60.62664041749151|[61.8465551556595...|
|   AAT| 33.56090773053562| 33.56303699613469|33.872553837658714| 33.23921960242961|[33.5609077305356...|
|  ABMD|24.849846952245322|24.871593335058844|25.358382593503315|24.334952478322702|[24.8498469522453...|
|   AIV| 33.79793227104989| 33.80576316672244| 34.12653463602813|33.449152063066016|[33.7979322710498...|
|  ARAY| 7.613426659528919|7.6105299785867215|7.7798464668094205| 7.438157316202708|[7.61342665952891...|
|  ACCO|11.012743428931586|11.017502363693659|

In [22]:
final_data = result.select("Independent", "avg(close)")
final_data.show()

+--------------------+------------------+
|         Independent|        avg(close)|
+--------------------+------------------+
|[5.15959507364142...| 5.167712239715598|
|[61.8465551556595...| 61.80857145942056|
|[33.5609077305356...| 33.56303699613469|
|[24.8498469522453...|24.871593335058844|
|[33.7979322710498...| 33.80576316672244|
|[7.61342665952891...|7.6105299785867215|
|[11.0127434289315...|11.017502363693659|
|[31.7210526258166...|31.733457472187798|
|[28.9755272907053...|28.974222148978274|
|[59.6247833474936...|  59.6699762107052|
|[27.1538259641326...|27.172286938581088|
|[23.2234534195658...|23.221512637526104|
|[39.0790911680911...|39.124330484330464|
|[13.2826600363156...|13.285934087157466|
|[3.72383008702859...|3.7142337339411506|
|[16.3256455387615...| 16.32278376401757|
|[101.233436648936...|101.19447202127645|
|[4.99635502364065...| 4.986302482269509|
|[29.5251366888369...|29.540100647132913|
|[47.1367517682828...| 47.15096415426471|
+--------------------+------------

In [23]:
from pyspark.ml.stat import Correlation

matrix = Correlation.corr(final_data, 'Independent')
cor_np = matrix.collect()[0][matrix.columns[0]].toArray()
cor_np

array([[1.        , 0.99999997, 0.99999997],
       [0.99999997, 1.        , 0.99999988],
       [0.99999997, 0.99999988, 1.        ]])

Dividir el dataset en train y test en proporcion de 75% y 25%

In [26]:
train, test = final_data.randomSplit([0.75, 0.25])

In [27]:
#Regresión lineal
lr = LinearRegression(featuresCol = 'Independent', labelCol="avg(close)", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.3305940591048401,0.329540860063884,0.33408348998765974]
Intercept: 0.19172031774823473


## Evaluación del modelo

In [28]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.278268
r2: 0.999977


In [29]:
train.describe().show()

+-------+------------------+
|summary|        avg(close)|
+-------+------------------+
|  count|              2389|
|   mean| 31.73779699874765|
| stddev|  58.6580902881375|
|    min|0.9262867559125384|
|    max|2149.1428571428573|
+-------+------------------+

